In [ ]:
# ============================================================================
# 한국전력공사 변동계수 알고리즘 - 완전한 버전
# 누락된 메서드들을 모두 포함한 실행 가능한 코드
# ============================================================================

import pandas as pd
import numpy as np
import json
import os
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import joblib
from datetime import datetime
from scipy import stats
warnings.filterwarnings('ignore')

class KEPCOVolatilityCoefficient:
    """
    한국전력공사 전력 사용패턴 변동계수 스태킹 알고리즘
    완전한 구현 버전
    """
    
    def __init__(self, results_path='./analysis_results'):
        """
        초기화
        Args:
            results_path: 1-2단계 결과가 저장된 폴더 경로
        """
        self.results_path = results_path
        
        # 기본 설정
        self.config = {
            'temporal_weights': {
                'peak_hours': [9, 10, 11, 14, 15, 18, 19, 20],  
                'peak_weight': 1.5,                               
                'off_peak_weight': 0.8                            
            },
            'seasonal_adjustment': {
                'summer_months': [6, 7, 8],      
                'winter_months': [12, 1, 2],     
                'summer_factor': 1.2,            
                'winter_factor': 1.1             
            },
            'industry_baselines': {
                '222': 0.25,  # 일반용(갑)‖고압A
                '226': 0.30,  # 일반용(을) 고압A  
                '311': 0.35,  # 산업용(갑) 저압
                '322': 0.20,  # 산업용(갑)‖고압A
                '726': 0.28   # 산업용(을) 고압A
            },
            'usage_type_factors': {
                '02': 1.1,    # 상업용
                '09': 0.9     # 광공업용
            },
            'anomaly_thresholds': {
                'cv_extreme': 1.0,           
                'zero_ratio_max': 0.1,       
                'night_day_ratio_max': 0.8   
            }
        }
        
        # 모델 구성요소
        self.level0_models = {}
        self.level1_model = None
        self.scalers = {}
        self.is_fitted = False
        
        # 과적합 방지 설정
        self.cv_folds = 5
        self.random_state = 42
        
        print("🔧 한국전력공사 변동계수 알고리즘 초기화")
        print(f"결과 폴더: {self.results_path}")
        
        # 스태킹 모델 초기화
        self._initialize_stacking_models()
        
        # 1-2단계 결과 자동 로드
        self._load_previous_results()
    
    def _initialize_stacking_models(self):
        """스태킹 모델 초기화"""
        # Level-0 모델들 (다양한 기법으로 변동성 측정)
        self.level0_models = {
            'rf_temporal': RandomForestRegressor(
                n_estimators=100, 
                max_depth=10, 
                random_state=self.random_state
            ),
            'gbm_seasonal': GradientBoostingRegressor(
                n_estimators=100, 
                max_depth=6, 
                random_state=self.random_state
            ),
            'ridge_basic': Ridge(alpha=1.0),
            'elastic_pattern': ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=self.random_state)
        }
        
        # Level-1 메타모델
        self.level1_model = Ridge(alpha=0.1)
        
        # 스케일러
        self.scalers = {
            'basic': StandardScaler(),
            'temporal': RobustScaler(),
            'seasonal': StandardScaler(),
            'pattern': RobustScaler(),
            'anomaly': StandardScaler()
        }
    
    def _load_previous_results(self):
        """1-2단계 결과 파일들을 자동으로 로드해서 설정 업데이트"""
        print("📂 1-2단계 결과 파일 로드 중...")
        
        try:
            # 1단계 결과 로드
            step1_file = os.path.join(self.results_path, 'analysis_results.json')
            if os.path.exists(step1_file):
                with open(step1_file, 'r', encoding='utf-8') as f:
                    step1_results = json.load(f)
                print("✅ 1단계 결과 로드 완료")
                self._update_config_from_step1(step1_results)
            else:
                print(f"⚠️ 1단계 결과 파일 없음: {step1_file}")
            
            # 2단계 결과 로드
            step2_file = os.path.join(self.results_path, 'volatility_summary.csv')
            if os.path.exists(step2_file):
                step2_results = pd.read_csv(step2_file)
                print("✅ 2단계 결과 로드 완료")
                self._update_config_from_step2(step2_results)
            else:
                print(f"⚠️ 2단계 결과 파일 없음: {step2_file}")
                
        except Exception as e:
            print(f"⚠️ 결과 파일 로드 중 오류: {e}")
            print("기본 설정으로 진행합니다.")
    
    def _update_config_from_step1(self, step1_results):
        """1단계 결과로 설정 업데이트"""
        print("🔄 1단계 결과로 설정 업데이트...")
        
        customer_summary = step1_results.get('customer_summary', {})
        if customer_summary:
            total_customers = customer_summary.get('total_customers', 3000)
            print(f"   고객 수: {total_customers:,}명")
            
            contract_types = customer_summary.get('contract_types', {})
            if contract_types:
                print(f"   계약종별: {len(contract_types)}개 유형")
                
                total_contracts = sum(contract_types.values())
                for contract, count in contract_types.items():
                    if str(contract) in self.config['industry_baselines']:
                        ratio = count / total_contracts
                        if ratio > 0.3:  
                            self.config['industry_baselines'][str(contract)] *= 0.95
    
    def _update_config_from_step2(self, step2_results):
        """2단계 결과로 설정 업데이트"""
        print("🔄 2단계 결과로 설정 업데이트...")
        
        for _, row in step2_results.iterrows():
            metric = row['metric']
            value = row['value']
            
            if metric == 'overall_cv':
                baseline_adjustment = value / 0.25  
                for contract_type in self.config['industry_baselines']:
                    self.config['industry_baselines'][contract_type] *= baseline_adjustment
                print(f"   전체 CV 기준 조정: {baseline_adjustment:.3f}배")
                
            elif metric == 'weekday_cv' and 'weekend_cv' in step2_results['metric'].values:
                weekend_row = step2_results[step2_results['metric'] == 'weekend_cv']
                if not weekend_row.empty:
                    weekend_cv = weekend_row['value'].iloc[0]
                    weekend_factor = weekend_cv / value if value > 0 else 1.0
                    self.config['weekend_factor'] = weekend_factor
                    print(f"   주말 팩터: {weekend_factor:.3f}")
        
        print("✅ 설정 업데이트 완료")
    
    def load_actual_data_for_features(self):
        """실제 LP 데이터를 로드해서 특성 생성"""
        print("📊 실제 데이터로 특성 생성...")
        
        try:
            # 전처리된 데이터가 있는지 확인
            processed_data_file = os.path.join(self.results_path, 'processed_lp_data.csv')
            
            if os.path.exists(processed_data_file):
                print("📂 전처리된 LP 데이터 로드...")
                lp_data = pd.read_csv(processed_data_file)
                lp_data['LP 수신일자'] = pd.to_datetime(lp_data['LP 수신일자'])
            else:
                # 원본 LP 데이터 로드
                print("📂 원본 LP 데이터 로드...")
                lp_data = self._load_original_lp_data()
            
            # 고객 데이터 로드
            customer_file = os.path.join(os.path.dirname(self.results_path), '제13회 산업부 공모전 대상고객.xlsx')
            if os.path.exists(customer_file):
                customer_data = pd.read_excel(customer_file)
            else:
                customer_data = None
                print("⚠️ 고객 데이터 파일 없음")
            
            # 특성 생성
            features_dict = self.create_features(lp_data, customer_data)
            
            print(f"✅ 특성 생성 완료: {len(features_dict)}명")
            return features_dict
            
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            print("샘플 데이터로 테스트 실행...")
            return self._create_sample_features()
    
    def _load_original_lp_data(self):
        """원본 LP 데이터 로드"""
        import glob
        
        base_path = os.path.dirname(self.results_path)
        lp_files = glob.glob(os.path.join(base_path, 'processed_LPData_*.csv'))
        
        if not lp_files:
            raise FileNotFoundError("LP 데이터 파일을 찾을 수 없습니다.")
        
        dataframes = []
        for file_path in sorted(lp_files):
            df = pd.read_csv(file_path)
            
            # 컬럼명 표준화
            column_mapping = {
                'LP수신일자': 'LP 수신일자',
                '순방향유효전력': '순방향 유효전력',
                '대체고객번호': '대체고객번호'
            }
            
            for old_col, new_col in column_mapping.items():
                if old_col in df.columns:
                    df = df.rename(columns={old_col: new_col})
            
            dataframes.append(df)
        
        lp_data = pd.concat(dataframes, ignore_index=True)
        lp_data['LP 수신일자'] = pd.to_datetime(lp_data['LP 수신일자'])
        
        return lp_data
    
    def _create_sample_features(self):
        """샘플 특성 데이터 생성 (테스트용)"""
        print("🧪 샘플 특성 데이터 생성...")
        
        sample_features = {}
        np.random.seed(42)
        
        for i in range(100):  
            customer_id = f'SAMPLE_{i:04d}'
            
            sample_features[customer_id] = {
                'basic': {
                    'cv_basic': np.random.normal(0.25, 0.1),
                    'range_volatility': np.random.normal(0.8, 0.3),
                    'iqr_volatility': np.random.normal(0.6, 0.2),
                    'skewness': np.random.normal(0.5, 0.3),
                    'kurtosis': np.random.normal(0.2, 0.5),
                    'mean_power': np.random.normal(50, 20),
                    'load_factor': np.random.normal(0.6, 0.2)
                },
                'temporal': {
                    'peak_cv': np.random.normal(0.3, 0.1),
                    'off_peak_cv': np.random.normal(0.2, 0.1),
                    'peak_mean': np.random.normal(60, 25),
                    'off_peak_mean': np.random.normal(40, 15),
                    'peak_off_peak_ratio': np.random.normal(1.5, 0.3),
                    'temporal_cv_diff': np.random.normal(0.1, 0.05),
                    'weekday_weekend_cv_ratio': np.random.normal(1.2, 0.3)
                },
                'seasonal': {
                    'summer_cv': np.random.normal(0.35, 0.15),
                    'winter_cv': np.random.normal(0.3, 0.12),
                    'summer_mean': np.random.normal(65, 30),
                    'winter_mean': np.random.normal(55, 25),
                    'seasonal_cv_diff': np.random.normal(0.05, 0.03),
                    'seasonal_stability': np.random.normal(0.1, 0.05)
                },
                'pattern': {
                    'daily_cv_stability': np.random.normal(0.05, 0.02),
                    'daily_cv_mean': np.random.normal(0.25, 0.1),
                    'autocorrelation': np.random.normal(0.7, 0.2),
                    'trend_volatility': np.random.normal(0.1, 0.05)
                },
                'anomaly': {
                    'zero_ratio': np.random.beta(1, 10),
                    'sudden_change_ratio': np.random.beta(1, 20),
                    'night_day_ratio': np.random.normal(0.4, 0.2),
                    'outlier_ratio': np.random.beta(1, 30)
                },
                'customer': {
                    'baseline_cv': 0.25,
                    'usage_factor': np.random.choice([0.9, 1.1]),
                    'contract_power_log': np.random.normal(6.0, 1.0)
                }
            }
        
        return sample_features
    
    def create_features(self, lp_data, customer_data=None):
        """실제 LP 데이터로 특성 생성"""
        print("🔄 실제 데이터 특성 생성 중...")
        
        features_dict = {}
        customers = lp_data['대체고객번호'].unique()
        
        # 메모리 고려해서 최대 1000명으로 제한
        if len(customers) > 1000:
            customers = customers[:1000]
            print(f"   메모리 효율성을 위해 {len(customers)}명으로 제한")
        
        for i, customer in enumerate(customers):
            if (i + 1) % 100 == 0:
                print(f"   진행: {i+1}/{len(customers)} ({(i+1)/len(customers)*100:.1f}%)")
            
            # 컬럼명 확인 및 표준화
            power_col = None
            for col in ['순방향 유효전력', '순방향유효전력']:
                if col in lp_data.columns:
                    power_col = col
                    break
            
            if power_col is None:
                print("⚠️ 전력 데이터 컬럼을 찾을 수 없습니다.")
                continue
            
            customer_lp = lp_data[lp_data['대체고객번호'] == customer].copy()
            
            if len(customer_lp) < 96:  # 최소 1일 데이터 필요
                continue
            
            # 시간 관련 파생 변수
            customer_lp['시간'] = customer_lp['LP 수신일자'].dt.hour
            customer_lp['요일'] = customer_lp['LP 수신일자'].dt.weekday
            customer_lp['월'] = customer_lp['LP 수신일자'].dt.month
            customer_lp['주말여부'] = customer_lp['요일'].isin([5, 6])
            
            power_series = customer_lp[power_col]
            
            # 1. 기본 변동성 특성
            basic_features = self._calculate_basic_volatility(power_series)
            
            # 2. 시간대별 변동성 특성
            temporal_features = self._calculate_temporal_volatility(customer_lp, power_col)
            
            # 3. 계절성 변동성 특성  
            seasonal_features = self._calculate_seasonal_volatility(customer_lp, power_col)
            
            # 4. 패턴 안정성 특성
            pattern_features = self._calculate_pattern_stability(customer_lp, power_col)
            
            # 5. 이상 패턴 특성
            anomaly_features = self._calculate_anomaly_features(customer_lp, power_col)
            
            # 6. 고객 특성
            customer_features = {}
            if customer_data is not None:
                customer_info = customer_data[customer_data['고객번호'] == customer]
                if not customer_info.empty:
                    customer_features = self._get_customer_features(customer_info.iloc[0])
                else:
                    customer_features = {'baseline_cv': 0.25, 'usage_factor': 1.0, 'contract_power_log': 6.0}
            else:
                customer_features = {'baseline_cv': 0.25, 'usage_factor': 1.0, 'contract_power_log': 6.0}
            
            features_dict[customer] = {
                'basic': basic_features,
                'temporal': temporal_features,
                'seasonal': seasonal_features,
                'pattern': pattern_features,
                'anomaly': anomaly_features,
                'customer': customer_features
            }
        
        print(f"✅ 특성 생성 완료: {len(features_dict)}명")
        return features_dict
    
    def _calculate_basic_volatility(self, power_series):
        """기본 변동성 특성 계산"""
        if len(power_series) == 0 or power_series.mean() == 0:
            return {
                'cv_basic': 0, 'range_volatility': 0, 'iqr_volatility': 0,
                'skewness': 0, 'kurtosis': 0, 'mean_power': 0, 'load_factor': 0
            }
        
        mean_power = power_series.mean()
        std_power = power_series.std()
        
        # 변동계수
        cv_basic = std_power / mean_power if mean_power > 0 else 0
        
        # 범위 변동성
        range_volatility = (power_series.max() - power_series.min()) / mean_power if mean_power > 0 else 0
        
        # IQR 변동성
        q75, q25 = np.percentile(power_series, [75, 25])
        median_power = np.median(power_series)
        iqr_volatility = (q75 - q25) / median_power if median_power > 0 else 0
        
        # 왜도와 첨도
        skewness = stats.skew(power_series)
        kurtosis = stats.kurtosis(power_series)
        
        # 부하율 (평균/최대)
        load_factor = mean_power / power_series.max() if power_series.max() > 0 else 0
        
        return {
            'cv_basic': cv_basic,
            'range_volatility': range_volatility,
            'iqr_volatility': iqr_volatility,
            'skewness': skewness,
            'kurtosis': kurtosis,
            'mean_power': mean_power,
            'load_factor': load_factor
        }
    
    def _calculate_temporal_volatility(self, customer_lp, power_col):
        """시간대별 변동성 특성 계산"""
        # 피크/오프피크 구분
        peak_mask = customer_lp['시간'].isin(self.config['temporal_weights']['peak_hours'])
        
        peak_data = customer_lp[peak_mask][power_col]
        off_peak_data = customer_lp[~peak_mask][power_col]
        
        # 각각의 변동계수 계산
        peak_cv = peak_data.std() / peak_data.mean() if len(peak_data) > 0 and peak_data.mean() > 0 else 0
        off_peak_cv = off_peak_data.std() / off_peak_data.mean() if len(off_peak_data) > 0 and off_peak_data.mean() > 0 else 0
        
        # 평일/주말 구분
        weekday_data = customer_lp[~customer_lp['주말여부']][power_col]
        weekend_data = customer_lp[customer_lp['주말여부']][power_col]
        
        weekday_cv = weekday_data.std() / weekday_data.mean() if len(weekday_data) > 0 and weekday_data.mean() > 0 else 0
        weekend_cv = weekend_data.std() / weekend_data.mean() if len(weekend_data) > 0 and weekend_data.mean() > 0 else 0
        
        return {
            'peak_cv': peak_cv,
            'off_peak_cv': off_peak_cv,
            'peak_mean': peak_data.mean() if len(peak_data) > 0 else 0,
            'off_peak_mean': off_peak_data.mean() if len(off_peak_data) > 0 else 0,
            'peak_off_peak_ratio': (peak_data.mean() / off_peak_data.mean()) if len(off_peak_data) > 0 and off_peak_data.mean() > 0 else 1,
            'temporal_cv_diff': abs(peak_cv - off_peak_cv),
            'weekday_weekend_cv_ratio': (weekday_cv / weekend_cv) if weekend_cv > 0 else 1
        }
    
    def _calculate_seasonal_volatility(self, customer_lp, power_col):
        """계절성 변동성 특성 계산"""
        # 여름/겨울/기타 구분
        summer_mask = customer_lp['월'].isin(self.config['seasonal_adjustment']['summer_months'])
        winter_mask = customer_lp['월'].isin(self.config['seasonal_adjustment']['winter_months'])
        
        summer_data = customer_lp[summer_mask][power_col]
        winter_data = customer_lp[winter_mask][power_col]
        other_data = customer_lp[~(summer_mask | winter_mask)][power_col]
        
        summer_cv = summer_data.std() / summer_data.mean() if len(summer_data) > 0 and summer_data.mean() > 0 else 0
        winter_cv = winter_data.std() / winter_data.mean() if len(winter_data) > 0 and winter_data.mean() > 0 else 0
        
        return {
            'summer_cv': summer_cv,
            'winter_cv': winter_cv,
            'summer_mean': summer_data.mean() if len(summer_data) > 0 else 0,
            'winter_mean': winter_data.mean() if len(winter_data) > 0 else 0,
            'seasonal_cv_diff': abs(summer_cv - winter_cv),
            'seasonal_stability': 1 - (abs(summer_cv - winter_cv) / max(summer_cv, winter_cv)) if max(summer_cv, winter_cv) > 0 else 1
        }
    
    def _calculate_pattern_stability(self, customer_lp, power_col):
        """패턴 안정성 특성 계산"""
        power_series = customer_lp[power_col]
        
        # 일별 변동계수 계산
        customer_lp['날짜'] = customer_lp['LP 수신일자'].dt.date
        daily_cvs = []
        
        for date in customer_lp['날짜'].unique():
            daily_data = customer_lp[customer_lp['날짜'] == date][power_col]
            if len(daily_data) > 1 and daily_data.mean() > 0:
                daily_cv = daily_data.std() / daily_data.mean()
                daily_cvs.append(daily_cv)
        
        daily_cv_stability = np.std(daily_cvs) if len(daily_cvs) > 1 else 0
        daily_cv_mean = np.mean(daily_cvs) if len(daily_cvs) > 0 else 0
        
        # 자기상관
        if len(power_series) > 1:
            autocorr = power_series.autocorr(lag=1)
            autocorr = autocorr if not np.isnan(autocorr) else 0
        else:
            autocorr = 0
        
        # 추세 변동성
        if len(power_series) > 2:
            trend = np.polyfit(range(len(power_series)), power_series, 1)[0]
            detrended = power_series - (trend * np.arange(len(power_series)))
            trend_volatility = detrended.std() / power_series.mean() if power_series.mean() > 0 else 0
        else:
            trend_volatility = 0
        
        return {
            'daily_cv_stability': daily_cv_stability,
            'daily_cv_mean': daily_cv_mean,
            'autocorrelation': autocorr,
            'trend_volatility': trend_volatility
        }
    
    def _calculate_anomaly_features(self, customer_lp, power_col):
        """이상 패턴 특성 계산"""
        power_series = customer_lp[power_col]
        
        # 0값 비율
        zero_ratio = (power_series == 0).sum() / len(power_series)
        
        # 급격한 변화 비율
        if len(power_series) > 1:
            changes = np.abs(power_series.diff())
            sudden_changes = changes > (changes.mean() + 2 * changes.std())
            sudden_change_ratio = sudden_changes.sum() / len(power_series)
        else:
            sudden_change_ratio = 0
        
        # 야간/주간 비율
        night_mask = customer_lp['시간'].isin([22, 23, 0, 1, 2, 3, 4, 5])
        day_mask = customer_lp['시간'].isin([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])
        
        night_mean = customer_lp[night_mask][power_col].mean() if night_mask.sum() > 0 else 0
        day_mean = customer_lp[day_mask][power_col].mean() if day_mask.sum() > 0 else 0
        
        night_day_ratio = night_mean / day_mean if day_mean > 0 else 0
        
        # 이상치 비율 (IQR 방법)
        q75, q25 = np.percentile(power_series, [75, 25])
        iqr = q75 - q25
        outlier_mask = (power_series < q25 - 1.5*iqr) | (power_series > q75 + 1.5*iqr)
        outlier_ratio = outlier_mask.sum() / len(power_series)
        
        return {
            'zero_ratio': zero_ratio,
            'sudden_change_ratio': sudden_change_ratio,
            'night_day_ratio': night_day_ratio,
            'outlier_ratio': outlier_ratio
        }
    
    def _get_customer_features(self, customer_info):
        """고객 정보에서 특성 추출"""
        # 계약종별에 따른 기준 변동계수
        contract_type = str(customer_info.get('계약종별', '322'))
        baseline_cv = self.config['industry_baselines'].get(contract_type, 0.25)
        
        # 용도별 팩터
        usage_type = str(customer_info.get('용도별', '09'))
        usage_factor = self.config['usage_type_factors'].get(usage_type, 1.0)
        
        # 계약전력 (로그 변환)
        contract_power = customer_info.get('계약전력', 100)
        contract_power_log = np.log10(max(contract_power, 1))
        
        return {
            'baseline_cv': baseline_cv,
            'usage_factor': usage_factor,
            'contract_power_log': contract_power_log
        }
    
    def _prepare_feature_matrix(self, features_dict):
        """특성 딕셔너리를 모델 학습용 행렬로 변환"""
        feature_matrices = {}
        customer_ids = list(features_dict.keys())
        
        # 각 카테고리별 특성 행렬 생성
        for category in ['basic', 'temporal', 'seasonal', 'pattern', 'anomaly', 'customer']:
            category_features = []
            
            for customer_id in customer_ids:
                customer_features = features_dict[customer_id][category]
                feature_vector = list(customer_features.values())
                category_features.append(feature_vector)
            
            feature_matrices[category] = np.array(category_features)
            
            # NaN 처리
            feature_matrices[category] = np.nan_to_num(feature_matrices[category], 0)
        
        return feature_matrices, customer_ids
    
    def fit(self, features_dict):
        """스태킹 모델 학습"""
        print("🚀 스태킹 모델 학습 시작...")
        
        # 특성 행렬 준비
        feature_matrices, customer_ids = self._prepare_feature_matrix(features_dict)
        
        # 타겟 변수 생성 (기본 변동계수를 기준으로)
        y_target = feature_matrices['basic'][:, 0]  # cv_basic
        
        print(f"학습 데이터: {len(customer_ids)}명")
        print(f"타겟 변수 범위: {y_target.min():.3f} ~ {y_target.max():.3f}")
        
        # Level-0 모델 학습 및 예측
        level0_predictions = {}
        
        for category, model_name in [
            ('temporal', 'rf_temporal'),
            ('seasonal', 'gbm_seasonal'), 
            ('basic', 'ridge_basic'),
            ('pattern', 'elastic_pattern')
        ]:
            print(f"  Level-0 모델 학습: {model_name} ({category})")
            
            X = feature_matrices[category]
            
            # 스케일링
            if category in self.scalers:
                X_scaled = self.scalers[category].fit_transform(X)
            else:
                X_scaled = X
            
            # 교차검증을 통한 Level-0 예측 생성
            cv = TimeSeriesSplit(n_splits=self.cv_folds)
            predictions = np.zeros(len(y_target))
            
            for train_idx, val_idx in cv.split(X_scaled):
                X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
                y_train, y_val = y_target[train_idx], y_target[val_idx]
                
                # 모델 학습
                model = self.level0_models[model_name]
                model.fit(X_train, y_train)
                
                # 검증 세트 예측
                val_pred = model.predict(X_val)
                predictions[val_idx] = val_pred
            
            level0_predictions[model_name] = predictions
            
            # 전체 데이터로 최종 모델 재학습
            self.level0_models[model_name].fit(X_scaled, y_target)
        
        # Level-1 메타 특성 구성
        meta_features = np.column_stack([
            level0_predictions['rf_temporal'],
            level0_predictions['gbm_seasonal'],
            level0_predictions['ridge_basic'],
            level0_predictions['elastic_pattern'],
            feature_matrices['anomaly'].mean(axis=1),  # 이상 패턴 요약
            feature_matrices['customer'][:, 0]         # baseline_cv
        ])
        
        # Level-1 모델 학습
        print("  Level-1 메타모델 학습...")
        self.level1_model.fit(meta_features, y_target)
        
        # 성능 평가
        final_predictions = self.level1_model.predict(meta_features)
        mae = mean_absolute_error(y_target, final_predictions)
        rmse = np.sqrt(mean_squared_error(y_target, final_predictions))
        r2 = r2_score(y_target, final_predictions)
        
        print(f"✅ 학습 완료 - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
        
        self.is_fitted = True
        
        return {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'level0_predictions': level0_predictions,
            'meta_features_shape': meta_features.shape
        }
    
    def predict(self, features_dict):
        """변동계수 예측"""
        if not self.is_fitted:
            raise ValueError("모델이 학습되지 않았습니다. fit() 메서드를 먼저 실행하세요.")
        
        print("🔮 변동계수 예측 중...")
        
        # 특성 행렬 준비
        feature_matrices, customer_ids = self._prepare_feature_matrix(features_dict)
        
        # Level-0 예측
        level0_predictions = {}
        
        for category, model_name in [
            ('temporal', 'rf_temporal'),
            ('seasonal', 'gbm_seasonal'),
            ('basic', 'ridge_basic'),
            ('pattern', 'elastic_pattern')
        ]:
            X = feature_matrices[category]
            
            # 스케일링 (학습시 사용한 스케일러 적용)
            if category in self.scalers:
                X_scaled = self.scalers[category].transform(X)
            else:
                X_scaled = X
            
            # 예측
            predictions = self.level0_models[model_name].predict(X_scaled)
            level0_predictions[model_name] = predictions
        
        # Level-1 메타 특성 구성
        meta_features = np.column_stack([
            level0_predictions['rf_temporal'],
            level0_predictions['gbm_seasonal'],
            level0_predictions['ridge_basic'],
            level0_predictions['elastic_pattern'],
            feature_matrices['anomaly'].mean(axis=1),
            feature_matrices['customer'][:, 0]
        ])
        
        # 최종 예측
        final_predictions = self.level1_model.predict(meta_features)
        
        # 예측 결과 딕셔너리 구성
        predictions_dict = {}
        for i, customer_id in enumerate(customer_ids):
            # 신뢰도 계산 (Level-0 모델들의 예측 일치도)
            level0_values = [
                level0_predictions['rf_temporal'][i],
                level0_predictions['gbm_seasonal'][i],
                level0_predictions['ridge_basic'][i],
                level0_predictions['elastic_pattern'][i]
            ]
            confidence = 1 - (np.std(level0_values) / max(np.mean(level0_values), 0.01))
            confidence = max(0, min(1, confidence))  # 0-1 범위로 제한
            
            predictions_dict[customer_id] = {
                'volatility_coefficient': max(0, final_predictions[i]),  # 음수 방지
                'confidence_score': confidence,
                'level0_predictions': {
                    'temporal': level0_predictions['rf_temporal'][i],
                    'seasonal': level0_predictions['gbm_seasonal'][i],
                    'basic': level0_predictions['ridge_basic'][i],
                    'pattern': level0_predictions['elastic_pattern'][i]
                }
            }
        
        print(f"✅ 예측 완료: {len(predictions_dict)}명")
        return predictions_dict
    
    def classify_volatility_grade(self, volatility_coefficient, customer_features=None):
        """변동성 등급 분류"""
        # 기본 기준값
        baseline_cv = 0.25
        
        # 고객별 기준값 조정
        if customer_features and 'customer' in customer_features:
            baseline_cv = customer_features['customer'].get('baseline_cv', 0.25)
        
        # 상대 변동계수 계산
        relative_cv = volatility_coefficient / baseline_cv
        
        # 등급 분류
        if relative_cv <= 0.8:
            grade = "매우 안정"
            risk_level = "낮음"
            recommendation = "현재 안정적인 전력 사용 패턴을 유지하세요."
        elif relative_cv <= 1.2:
            grade = "안정"
            risk_level = "낮음"
            recommendation = "양호한 전력 사용 패턴입니다."
        elif relative_cv <= 1.8:
            grade = "보통"
            risk_level = "중간"
            recommendation = "전력 사용 패턴 최적화를 고려해보세요."
        elif relative_cv <= 2.5:
            grade = "불안정"
            risk_level = "높음"
            recommendation = "전력 사용 패턴 개선이 필요합니다."
        else:
            grade = "매우 불안정"
            risk_level = "매우 높음"
            recommendation = "즉시 전력 사용 패턴 점검 및 개선이 필요합니다."
        
        return {
            'grade': grade,
            'risk_level': risk_level,
            'relative_cv': relative_cv,
            'baseline_cv': baseline_cv,
            'recommendation': recommendation
        }
    
    def generate_report(self, predictions, save_path=None):
        """종합 리포트 생성"""
        print("📋 종합 리포트 생성 중...")
        
        # 기본 통계
        cv_values = [pred['volatility_coefficient'] for pred in predictions.values()]
        confidence_scores = [pred['confidence_score'] for pred in predictions.values()]
        
        # 등급별 분포
        grade_distribution = {}
        for customer_id, pred in predictions.items():
            grade_info = self.classify_volatility_grade(pred['volatility_coefficient'])
            grade = grade_info['grade']
            grade_distribution[grade] = grade_distribution.get(grade, 0) + 1
        
        # 리포트 구성
        report = {
            'analysis_summary': {
                'total_customers': len(predictions),
                'analysis_date': datetime.now().isoformat(),
                'model_type': 'Stacking Ensemble'
            },
            'volatility_statistics': {
                'mean_cv': np.mean(cv_values),
                'std_cv': np.std(cv_values),
                'min_cv': np.min(cv_values),
                'max_cv': np.max(cv_values),
                'median_cv': np.median(cv_values),
                'percentiles': {
                    'p25': np.percentile(cv_values, 25),
                    'p75': np.percentile(cv_values, 75),
                    'p90': np.percentile(cv_values, 90),
                    'p95': np.percentile(cv_values, 95)
                }
            },
            'confidence_statistics': {
                'mean_confidence': np.mean(confidence_scores),
                'min_confidence': np.min(confidence_scores),
                'max_confidence': np.max(confidence_scores)
            },
            'grade_distribution': grade_distribution,
            'high_risk_customers': [],
            'recommendations': {
                'immediate_attention': [],
                'monitoring_required': [],
                'stable_customers': []
            }
        }
        
        # 고위험 고객 식별
        for customer_id, pred in predictions.items():
            cv = pred['volatility_coefficient']
            grade_info = self.classify_volatility_grade(cv)
            
            if grade_info['risk_level'] in ['높음', '매우 높음']:
                report['high_risk_customers'].append({
                    'customer_id': customer_id,
                    'volatility_coefficient': cv,
                    'grade': grade_info['grade'],
                    'risk_level': grade_info['risk_level'],
                    'confidence': pred['confidence_score']
                })
            
            # 권장사항 분류
            if grade_info['risk_level'] == '매우 높음':
                report['recommendations']['immediate_attention'].append(customer_id)
            elif grade_info['risk_level'] in ['높음', '중간']:
                report['recommendations']['monitoring_required'].append(customer_id)
            else:
                report['recommendations']['stable_customers'].append(customer_id)
        
        # 파일 저장
        if save_path:
            with open(save_path, 'w', encoding='utf-8') as f:
                json.dump(report, f, ensure_ascii=False, indent=2)
            print(f"💾 리포트 저장: {save_path}")
        
        return report
    
    def save_model(self, file_path):
        """모델 저장"""
        if not self.is_fitted:
            print("⚠️ 학습되지 않은 모델은 저장할 수 없습니다.")
            return False
        
        try:
            model_data = {
                'level0_models': self.level0_models,
                'level1_model': self.level1_model,
                'scalers': self.scalers,
                'config': self.config,
                'is_fitted': self.is_fitted
            }
            
            joblib.dump(model_data, file_path)
            print(f"💾 모델 저장 완료: {file_path}")
            return True
            
        except Exception as e:
            print(f"❌ 모델 저장 실패: {e}")
            return False
    
    def load_model(self, file_path):
        """모델 로드"""
        try:
            model_data = joblib.load(file_path)
            
            self.level0_models = model_data['level0_models']
            self.level1_model = model_data['level1_model']
            self.scalers = model_data['scalers']
            self.config = model_data['config']
            self.is_fitted = model_data['is_fitted']
            
            print(f"✅ 모델 로드 완료: {file_path}")
            return True
            
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return False
    
    def run_complete_analysis(self):
        """전체 변동계수 분석 실행"""
        start_time = datetime.now()
        
        print("🚀 변동계수 알고리즘 분석 시작")
        print(f"시작 시간: {start_time}")
        
        try:
            # 1. 특성 데이터 준비
            features_dict = self.load_actual_data_for_features()
            
            if not features_dict:
                print("❌ 특성 데이터 준비 실패")
                return False
            
            # 2. 모델 학습
            print("🚀 스태킹 모델 학습...")
            training_results = self.fit(features_dict)
            
            # 3. 예측 수행
            print("🔮 변동계수 예측...")
            predictions = self.predict(features_dict)
            
            # 4. 결과 분석 및 리포트
            print("📋 최종 리포트 생성...")
            report_path = os.path.join(self.results_path, 'volatility_coefficient_report.json')
            final_report = self.generate_report(predictions, save_path=report_path)
            
            # 5. 모델 저장
            model_path = os.path.join(self.results_path, 'kepco_volatility_model.pkl')
            self.save_model(model_path)
            
            # 6. 등급별 분류 결과 저장
            self._save_classification_results(predictions)
            
            end_time = datetime.now()
            duration = end_time - start_time
            
            print("\n" + "="*60)
            print("🏆 변동계수 분석 완료!")
            print("="*60)
            print(f"소요 시간: {duration}")
            print(f"분석 고객: {len(predictions)}명")
            print(f"평균 변동계수: {np.mean([p['volatility_coefficient'] for p in predictions.values()]):.4f}")
            
            # 등급별 분포 출력
            grades = {}
            for pred in predictions.values():
                grade_info = self.classify_volatility_grade(pred['volatility_coefficient'])
                grade = grade_info['grade']
                grades[grade] = grades.get(grade, 0) + 1
            
            print("\n🎯 변동성 등급별 분포:")
            for grade, count in grades.items():
                pct = count / len(predictions) * 100
                print(f"   {grade}: {count}명 ({pct:.1f}%)")
            
            print(f"\n📁 결과 파일:")
            print(f"   ✅ 모델: {model_path}")
            print(f"   ✅ 리포트: {report_path}")
            print(f"   ✅ 분류결과: {os.path.join(self.results_path, 'customer_volatility_grades.csv')}")
            
            return True
            
        except Exception as e:
            print(f"❌ 분석 실패: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def _save_classification_results(self, predictions):
        """고객별 변동성 등급 분류 결과 저장"""
        results = []
        
        for customer_id, pred in predictions.items():
            grade_info = self.classify_volatility_grade(pred['volatility_coefficient'])
            
            results.append({
                '고객번호': customer_id,
                '변동계수': pred['volatility_coefficient'],
                '변동성등급': grade_info['grade'],
                '위험수준': grade_info['risk_level'],
                '상대변동계수': grade_info['relative_cv'],
                '기준변동계수': grade_info['baseline_cv'],
                '신뢰도': pred['confidence_score'],
                '권장사항': grade_info['recommendation']
            })
        
        results_df = pd.DataFrame(results)
        output_path = os.path.join(self.results_path, 'customer_volatility_grades.csv')
        results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"💾 분류 결과 저장: {output_path}")

# 실행 함수
def main():
    """메인 실행 함수"""
    print("=" * 60)
    print("한국전력공사 변동계수 알고리즘 - 완전한 버전")
    print("=" * 60)
    
    # 결과 폴더 확인
    results_folders = ['./analysis_results', './results', './output']
    results_path = None
    
    for folder in results_folders:
        if os.path.exists(folder):
            results_path = folder
            break
    
    if results_path is None:
        print("⚠️ 결과 폴더를 찾을 수 없습니다.")
        print("다음 중 하나의 폴더에 1-2단계 결과를 저장하세요:")
        for folder in results_folders:
            print(f"   - {folder}")
        results_path = './analysis_results'
        os.makedirs(results_path, exist_ok=True)
        print(f"✅ 기본 폴더 생성: {results_path}")
    
    # 알고리즘 실행
    algorithm = KEPCOVolatilityCoefficient(results_path=results_path)
    success = algorithm.run_complete_analysis()
    
    if success:
        print("\n🎉 변동계수 분석이 성공적으로 완료되었습니다!")
    else:
        print("\n💥 변동계수 분석 중 오류가 발생했습니다.")
    
    return algorithm

if __name__ == "__main__":
    main()